# Machine Reading: Advanced Topics in Word Vectors
## Part III. Pre-trained Models and Extended Vector Algorithms (50 mins)

This is a 4-part series of Jupyter notebooks on the topic of word embeddings for the Digital Humanities. Each part is comprised of a mix of theoretical explanations and fill-in-the-blanks activities of increasing difficulty.

Instructors:
- Eun Seo Jo, <a href="mailto:eunseo@stanford.edu">*eunseo@stanford.edu*</a>, Stanford University
- Javier de la Rosa, <a href="mailto:versae@stanford.edu">*versae@stanford.edu*</a>, Stanford University
- Scott Bailey, <a href="mailto:scottbailey@stanford.edu">*scottbailey@stanford.edu*</a>, Stanford University

This unit will explore the various flavors of word embeddings specifically tailored to sentences, word meaning, paragraph, or entire documents. We will give an overview of pre-trained embeddings including where they can be found and how to use them and what for.

- 0:00 - 0:20 Pre-trained word embeddings (where to find them, which are good, configurations, trained corpus, etc., e.g. https://github.com/facebookresearch/fastText)
- 0:20 - 0:35 Overview of other 2Vecs & other vector engineering: Paragraph2Vec, Sense2Vec, Doc2Vec, etc.
- 0:35 - 0:50 [Activity 3] Choose, download, and use a pre-trained model

### 0. Setting Up 

Before we get started, let's go ahead and set up our notebook. We will start by importing a few Python libraries that we will use throughout the workshop.

#### What are these libraries?

1. NumPy: This is package for scientific computing in python. For us, NumPy is useful for vector operations. 
2. NLTK: Easy to use python package for text processing (lemmatization, tokenization, POS-tagging, etc.)
3. matplotlib, seaborn, and Plotly: Plotting packages for visualization
4. sciKit-learn: Easy to use python package for machine learning algorithms and preprocessing tools
5. gensim: Builtin word2vec and other NLP algorithms
5. fastText: Super fast word embeddings library

We will be working with a few sample texts using NLTK's corpus package.

In [12]:
%%capture --no-stderr
import sys
!pip install Cython
!pip install numpy nltk scikit-learn matplotlib gensim seaborn plotly fasttext
!python -m nltk.downloader all
print("All done!", file=sys.stderr)

All done!


If all went well, we should be able now to import the next packages into our workspace

In [13]:
import numpy as np
import nltk
# import plotly.plotly as py
import sklearn
import matplotlib.pyplot as plt
import gensim
import fasttext

from IPython.display import HTML



---



### 1. Pre-trained word embeddings

#### Where to find them, which are good, configurations, trained corpus, etc., e.g. https://github.com/facebookresearch/fastText)



In [3]:
HTML("""
<div style="font-size: 1em; margin: 1em 0 1em 0; border: 1px solid #86989B; background-color: #f7f7f7; padding: 0;">
<p style="margin: 0; padding: 0.1em 0 0.1em 0.5em; color: white; border-bottom: 1px solid #86989B; font-weight: bold; background-color: #AFC1C4;">
Activity
</p>
<p style="margin: 0.5em 1em 0.5em 1em; padding: 0;">
Your turn: Generate a vector including integers from 4 and 8 of size 10
<br>
<em>
<strong>Hint</strong>: Use the numpy functions
</em>
</p>
</div>
""")

Solution:
```python
np.random.randint(...)
```

In [2]:
# Enter your code here

A **document-term matrix** is just a big table (formally, a mathematical matrix) that describes the frequency of words or terms that occur in a collection of documents. In a document-term matrix, **rows correspond to documents** in the collection and **columns correspond to terms**. In 

In our case, since we only have one document, our document-term matrix only has one row (doc `0`) and looks like this.

|             | also | is | of | piece | random | some | text | this |
| ----------- |:----:|:--:|:--:|:-----:|:------:|:----:|:----:|:----:|
| Document #1 |  2   | 2  | 1  | 1     |  1     |  1   |  3   |   2  |

Which can easily be extracted by using the `transform()` method our `CountVectorizer()`.

In this section we describe the datasets, embeddings, and word lists used, as well as how bias is quantified. More detail, including descriptions of additional embeddings and the full word lists, are in SI Appendix, section A. All of our data and code are available on GitHub (https://github.com/nikhgarg/EmbeddingDynamicStereotypes), and we link to external data sources as appropriate.

Embeddings.
This work uses several pretrained word embeddings publicly available online; refer to the respective sources for in-depth discussion of their training parameters. These embeddings are among the most commonly used English embeddings, vary in the datasets on which they were trained, and between them cover the best-known algorithms to construct embeddings. One finding in this work is that, although there is some heterogeneity, gender and ethnic bias is generally consistent across embeddings. Here we restrict descriptions to embeddings used in the main exposition. For consistency, only single words are used, all vectors are normalized by their l2 norm, and words are converted to lowercase.

Google News word2vec vectors.
Vectors trained on about 100 billion words in the Google News dataset (24, 25). Vectors are available at https://code.google.com/archive/p/word2vec/.

Google Books/COHA.
Vectors trained on a combined corpus of genre-balanced Google Books and the COHA (48) by the authors of ref. 26. For each decade, a separate embedding is trained from the corpus data corresponding to that decade. The dataset is specifically designed to enable comparisons across decades, and the creators take special care to avoid selection bias issues. The vectors are available at https://nlp.stanford.edu/projects/histwords/, and we limit our analysis to the SVD and skip-gram with negative sampling (SGNS) (also known as word2vec) embeddings in the 1900s. Note that the Google Books data may include some non-American sources and the external metrics we use are American. However, this does not appreciably affect results. In the main text, we exclusively use SGNS embeddings; results with SVD embeddings are in SI Appendix and are qualitatively similar to the SGNS results. Unless otherwise specified, COHA indicates these embeddings trained using the SGNS algorithm.

New York Times.
We train embeddings over time from The New York Times Annotated Corpus (28), using 1.8 million articles from the New York Times between 1988 and 2005. We use the GLoVe algorithm (27) and train embeddings over 3-y windows (so the 2000 embeddings, for example, contain articles from 1999 to 2001).

In SI Appendix we also use other embeddings available at https://nlp.stanford.edu/projects/glove/.

Models
- Word2Vec Model of ECCO, “Literature and Language,” 1700-99 (1.9 billion words; skip-gram size of 10 words): http://ryanheuser.org/data/word2vec.ECCO.skipgram_n=10.model.txt.gz
- Word2Vec Models for Twenty-year Periods of 18C (ECCO, “Literature and Language,” 1700-99) (150 million words each; skip-gram size of 10 words): https://archive.org/details/word-vectors-18c-word2vec-models-across-20-year-periods
- Word2Vec Model of ECCO-TCP, 1700-99 (80 million words; skip-gram size of 10 words): http://ryanheuser.org/data/word2vec.ECCO-TCP.skipgram_n=10.txt.zip
- Word2Vec Model of ECCO-TCP, 1700-99 (80 million words; skip-gram size of 5 words): http://ryanheuser.org/data/word2vec.ECCO-TCP.txt.zip
Code
Code to evaluate a word2vec model against the Miller Analogies Test
Code to produce a semantic network from a gensim word2vec model
Code for aligning two gensim word2vec models using Procrustes matrix alignment

Notes and links:
https://github.com/versae/word_vectors_dh2018
https://arxiv.org/pdf/1310.4546.pdf
http://www.pnas.org/content/early/2018/03/30/1720347115.full#sec-17
https://github.com/nikhgarg/EmbeddingDynamicStereotypes
http://ryanheuser.org/word-vectors/
http://ryanheuser.org/word2vec-vs-the-mat/
http://ahogrammer.com/2017/01/20/the-list-of-pretrained-word-embeddings/
https://github.com/Kyubyong/wordvectors
https://github.com/facebookresearch/fastText
https://pypi.org/project/fasttext/
https://gist.github.com/bhaettasch/d7f4e22e79df3c8b6c20
https://towardsdatascience.com/using-fasttext-and-svd-to-visualise-word-embeddings-instantly-5b8fa870c3d1
https://radimrehurek.com/gensim/models/fasttext.html#module-gensim.models.fasttext
https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/FastText_Tutorial.ipynb
https://towardsdatascience.com/word-embedding-with-word2vec-and-fasttext-a209c1d3e12c

https://docs.google.com/document/d/1nKEPA-jKvIkJyRhi2Ok_v3Hjnpv5pzsOhdSTSDNoYSo/edit?ts=5aa7ef5f